In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import gc

from sklearn.preprocessing import StandardScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
fig_width = 12
fig_height = 5
font_size = 15

In [ ]:
# Loading input data
#train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
#test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
#train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
#test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

# Loading input data
train_transaction = pd.read_parquet('/kaggle/input/train-transaction/train_transaction.parquet', engine='pyarrow')
test_transaction = pd.read_parquet('/kaggle/input/train-transaction/test_transaction.parquet', engine='pyarrow')
train_identity = pd.read_parquet('/kaggle/input/train-transaction/train_identity.parquet', engine='pyarrow')
test_identity = pd.read_parquet('/kaggle/input/train-transaction/test_identity.parquet', engine='pyarrow')

In [ ]:
train_set = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test_set = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [ ]:
#List items in each column
#for col, values in train_set.iteritems():
#    num_uniques = values.nunique()
#    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
#    print (values.unique())
#    print ('\n')
train_set.head(5)

In [ ]:
del train_transaction
del test_transaction
gc.collect()

In [ ]:
category_cols =['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2',
                'M1','M2','M3','M4','M5','M6','M7','M8','M9']

In [ ]:
#numeric_cols = []
#for col in all_columns:
#    if col not in category_cols:
#        numeric_cols.append(col)
time_delta_cols = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14','D15']
counter_cols = ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14']
numeric_cols = ['TransactionAmt','dist1','dist2']
label_col = ['isFraud']
text_col= ['P_emaildomain','R_emaildomain']

In [ ]:
#train_set['D1'].fillna(0,inplace=True)
for col in time_delta_cols:
    train_set[col].fillna(0,inplace=True)
    test_set[col].fillna(0,inplace=True)

In [ ]:
for col in counter_cols:
    train_set[col].fillna(0,inplace=True)
    test_set[col].fillna(0,inplace=True)

In [ ]:
for col in numeric_cols:
    train_set[col].fillna(0,inplace=True)
    test_set[col].fillna(0,inplace=True)

In [ ]:
train_set['Dsum'] = train_set['D1'] + train_set['D2'] + train_set['D3'] + train_set['D4'] + train_set['D5']+ train_set['D6'] + train_set['D7'] + train_set['D8']+ train_set['D9'] + train_set['D10'] + train_set['D11']+train_set['D12']+ train_set['D13'] + train_set['D14'] + train_set['D15']
test_set['Dsum'] = test_set['D1'] + test_set['D2'] + test_set['D3'] + test_set['D4'] + test_set['D5']+ test_set['D6'] + test_set['D7'] + test_set['D8']+ test_set['D9'] + test_set['D10'] + test_set['D11'] + test_set['D12']+ test_set['D13'] + test_set['D14'] + test_set['D15']


In [ ]:
train_set['amt_cat'] = pd.cut(train_set.TransactionAmt,bins=[0,50,100,150,200,100000],labels=['<=50','>50&<=100','>100&<=150','>150&<=200','>200'])

In [ ]:
train_set['Csum'] = train_set['C1'] + train_set['C2'] + train_set['C3'] + train_set['C4'] + train_set['C5']+ train_set['C6'] + train_set['C7'] + train_set['C8']+ train_set['C9'] + train_set['C10'] + train_set['C11']+train_set['C12']+ train_set['C13'] + train_set['C14']
test_set['Csum'] = test_set['C1'] + test_set['C2'] + test_set['C3'] + test_set['C4'] + test_set['C5']+ test_set['C6'] + test_set['C7'] + test_set['C8']+ test_set['C9'] + test_set['C10'] + test_set['C11'] + test_set['C12']+ test_set['C13'] + test_set['C14']

In [ ]:
#train_set['TransactionAmt'].isnull().sum()

In [ ]:
#train_set['C8'].describe()

In [ ]:
#for col in numeric_cols:
#    train_set[col] = train_set[col].fillna(train_set[col].mean(),inplace=True)
#    test_set[col] = test_set[col].fillna(test_set[col].mean(),inplace=True)

In [ ]:
category_cols =['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2',
                'M1','M2','M3','M4','M5','M6','M7','M8','M9']

In [ ]:
features_cols = label_col + time_delta_cols + counter_cols + numeric_cols + text_col + category_cols + ['Dsum'] + ['Csum']+ ['amt_cat']
features_cols

In [ ]:
train_set = train_set[features_cols]
#test_set = test_transaction[features_cols]

In [ ]:
train_set.head()

In [ ]:
train_set[category_cols] = train_set[category_cols].astype('str')#.fillna('na')

In [ ]:
train_set.isFraud.value_counts()

In [ ]:
isFraud_cnt = train_set.isFraud.value_counts()
isFraud_cnt.plot(subplots=True, kind='bar',figsize=(fig_width,fig_height), fontsize = font_size)

In [ ]:
isFraud_cnt = train_set.isFraud.value_counts(normalize=True)
isFraud_cnt.plot.pie(subplots=True, figsize=(fig_width,fig_height), fontsize = font_size, autopct='%1.1f%%')

In [ ]:
train_set_nonfraud = train_set[train_set['isFraud'] == 0]
train_set_nof_productCD = train_set_nonfraud['ProductCD'].value_counts().rename('Non-Fraud')
train_set_fraud = train_set[train_set['isFraud'] == 1]
train_set_fraud_productCD = train_set_fraud['ProductCD'].value_counts().rename('Fraud')

In [ ]:
dfx = pd.merge(train_set_nof_productCD, train_set_fraud_productCD, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Product CD (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 50})

In [ ]:
train_set_nof_card1 = train_set_nonfraud['card1'].value_counts().rename('Non-Fraud - Card1').rename('Non-Fraud')
train_set_fraud_card1 = train_set_fraud['card1'].value_counts().rename('Fraud - Card1').rename('Fraud')

In [ ]:
dfx = pd.merge(train_set_nof_card1[:10], train_set_fraud_card1[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Card1 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 50})

In [ ]:
train_set_nof_card2 = train_set_nonfraud['card2'].value_counts().rename('Non-Fraud - Card2')
train_set_fraud_card2 = train_set_fraud['card2'].value_counts().rename('Fraud - Card2')

dfx = pd.merge(train_set_nof_card2[:10], train_set_fraud_card2[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Card2 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_card3 = train_set_nonfraud['card3'].value_counts().rename('Non-Fraud - Card3')
train_set_fraud_card3 = train_set_fraud['card3'].value_counts().rename('Fraud - Card3')

dfx = pd.merge(train_set_nof_card3[:5], train_set_fraud_card3[:5], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 5 Card3 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_card4 = train_set_nonfraud['card4'].value_counts().rename('Non-Fraud - Card4')
train_set_fraud_card4 = train_set_fraud['card4'].value_counts().rename('Fraud - Card4')

dfx = pd.merge(train_set_nof_card4, train_set_fraud_card4, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Card4 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_card5 = train_set_nonfraud['card5'].value_counts().rename('Non-Fraud - Card5')
train_set_fraud_card5 = train_set_fraud['card5'].value_counts().rename('Fraud - Card5')

dfx = pd.merge(train_set_nof_card5[:10], train_set_fraud_card5[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Card5 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_card6 = train_set_nonfraud['card6'].value_counts().rename('Non-Fraud - Card6')
train_set_fraud_card6 = train_set_fraud['card6'].value_counts().rename('Fraud - Card6')

dfx = pd.merge(train_set_nof_card6[:10], train_set_fraud_card6[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Card6 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_addr1 = train_set_nonfraud['addr1'].value_counts().rename('Non-Fraud - addr1')
train_set_fraud_addr1 = train_set_fraud['addr1'].value_counts().rename('Fraud - addr1')

dfx = pd.merge(train_set_nof_addr1[:10], train_set_fraud_addr1[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Addr1(Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_addr1 = train_set_nonfraud[train_set_nonfraud['addr1']!='nan'].addr1.value_counts().rename('Non-Fraud - addr1')
train_set_fraud_addr1 = train_set_fraud[train_set_fraud['addr1']!='nan'].addr1.value_counts().rename('Fraud - addr1')

dfx = pd.merge(train_set_nof_addr1[:10], train_set_fraud_addr1[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Addr1 Excluded Nan (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_addr2 = train_set_nonfraud['addr2'].value_counts().rename('Non-Fraud - addr2')
train_set_fraud_addr2 = train_set_fraud['addr2'].value_counts().rename('Fraud - addr2')

dfx = pd.merge(train_set_nof_addr2[:10], train_set_fraud_addr2[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Addr2(Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_addr2 = train_set_nonfraud[train_set_nonfraud['addr2']!='nan'].addr2.value_counts().rename('Non-Fraud - addr2')
train_set_fraud_addr2 = train_set_fraud[train_set_fraud['addr2']!='nan'].addr2.value_counts().rename('Fraud - addr2')

dfx = pd.merge(train_set_nof_addr2[:10], train_set_fraud_addr2[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Addr2 Excluded Nan (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_addr2 = train_set_nonfraud[(train_set_nonfraud['addr2']!='nan') & (train_set_nonfraud['addr2']!='87.0') & (train_set_nonfraud['addr2']!='60.0')].addr2.value_counts().rename('Non-Fraud - addr2')
train_set_fraud_addr2 = train_set_fraud[(train_set_fraud['addr2']!='nan') & (train_set_fraud['addr2']!='87.0') & (train_set_fraud['addr2']!='60.0')].addr2.value_counts().rename('Fraud - addr2')

dfx = pd.merge(train_set_nof_addr2[:10], train_set_fraud_addr2[:10], right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Top 10 Addr2 Excluded Nan,87,60 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_p_edomain = train_set_nonfraud['P_emaildomain'].value_counts().rename('Non-Fraud - P_emaildomain')
train_set_fraud_p_edomain = train_set_fraud['P_emaildomain'].value_counts().rename('Fraud - P_emaildomain')

df_train_set_p_edomain = pd.merge(train_set_fraud_p_edomain, train_set_nof_p_edomain, right_index = True,
               left_index = True)
df_train_set_p_edomain[0:30].plot(subplots=True, kind='bar',figsize=(fig_width,fig_height), fontsize = font_size)


In [ ]:
train_set_nof_p_edomain[0:30].plot(kind='bar',figsize=(fig_width,fig_height),color='DarkOrange', fontsize = font_size, title = 'Top 30 non fraud p email domain in training set')

In [ ]:
train_set_fraud_p_edomain[0:30].plot(kind='bar',figsize=(fig_width,fig_height),fontsize = font_size, title = 'Top 30 fraud p email domain in training set')

In [ ]:
train_set_nof_r_edomain = train_set_nonfraud['R_emaildomain'].value_counts().rename('Non-Fraud - R_emaildomain')
train_set_fraud_r_edomain = train_set_fraud['R_emaildomain'].value_counts().rename('Fraud - R_emaildomain')

df_train_set_r_edomain = pd.merge(train_set_fraud_p_edomain, train_set_nof_p_edomain, right_index = True,
               left_index = True)
df_train_set_r_edomain[0:30].plot(subplots=True, kind='bar',figsize=(fig_width,fig_height), fontsize = font_size)


In [ ]:
train_set_nof_r_edomain[0:30].plot(kind='bar',figsize=(fig_width,fig_height),color='DarkOrange', fontsize = font_size, title = 'Top 30 non fraud R email domain in training set')

In [ ]:
train_set_fraud_r_edomain[0:30].plot(kind='bar',figsize=(fig_width,fig_height),fontsize = font_size, title = 'Top 30 fraud r email domain in training set')

In [ ]:
train_set_nonfraud['R_emaildomain'].value_counts(dropna=False)

In [ ]:
train_set_fraud['R_emaildomain'].value_counts(dropna=False)

In [ ]:
train_set_nonfraud['P_emaildomain'].value_counts(dropna=False)

In [ ]:
train_set_fraud['P_emaildomain'].value_counts(dropna=False)

In [ ]:
train_set_fraud['P_neq_R'] = train_set_fraud['P_emaildomain'] != train_set_fraud['R_emaildomain']
train_set_nonfraud['P_neq_R'] = train_set_nonfraud['P_emaildomain'] != train_set_nonfraud['R_emaildomain']


In [ ]:
train_set_fraud_pneqr = train_set_fraud['P_neq_R'].value_counts().rename('Fraud - P not equal R')
train_set_nof_pneqr = train_set_nonfraud['P_neq_R'].value_counts().rename('Non Fraud - P not equal R')

In [ ]:
dfx = pd.merge(train_set_nof_pneqr, train_set_fraud_pneqr, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="P_emaildomain not equal R_emaildomain  (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m1 = train_set_nonfraud['M1'].value_counts().rename('Non-Fraud - M1')
train_set_fraud_m1 = train_set_fraud['M1'].value_counts().rename('Fraud - M1')

dfx = pd.merge(train_set_nof_m1, train_set_fraud_m1, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M1  (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m2 = train_set_nonfraud['M2'].value_counts().rename('Non-Fraud - M2')
train_set_fraud_m2 = train_set_fraud['M2'].value_counts().rename('Fraud - M2')

dfx = pd.merge(train_set_nof_m2, train_set_fraud_m2, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M2 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m3 = train_set_nonfraud['M3'].value_counts().rename('Non-Fraud - M3')
train_set_fraud_m3 = train_set_fraud['M3'].value_counts().rename('Fraud - M3')

dfx = pd.merge(train_set_nof_m3, train_set_fraud_m3, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M3 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m4 = train_set_nonfraud['M4'].value_counts().rename('Non-Fraud - M4')
train_set_fraud_m4 = train_set_fraud['M4'].value_counts().rename('Fraud - M4')

dfx = pd.merge(train_set_nof_m4, train_set_fraud_m4, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M4 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m5 = train_set_nonfraud['M5'].value_counts().rename('Non-Fraud - M5')
train_set_fraud_m5 = train_set_fraud['M5'].value_counts().rename('Fraud - M5')

dfx = pd.merge(train_set_nof_m5, train_set_fraud_m5, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M5 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m6 = train_set_nonfraud['M6'].value_counts().rename('Non-Fraud - M6')
train_set_fraud_m6 = train_set_fraud['M6'].value_counts().rename('Fraud - M6')

dfx = pd.merge(train_set_nof_m6, train_set_fraud_m6, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M6 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m7 = train_set_nonfraud['M7'].value_counts().rename('Non-Fraud - M7')
train_set_fraud_m7 = train_set_fraud['M7'].value_counts().rename('Fraud - M7')

dfx = pd.merge(train_set_nof_m7, train_set_fraud_m7, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M6 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m8 = train_set_nonfraud['M8'].value_counts().rename('Non-Fraud - M8')
train_set_fraud_m8 = train_set_fraud['M8'].value_counts().rename('Fraud - M8')

dfx = pd.merge(train_set_nof_m8, train_set_fraud_m8, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M6 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nof_m9 = train_set_nonfraud['M9'].value_counts().rename('Non-Fraud - M9')
train_set_fraud_m9 = train_set_fraud['M9'].value_counts().rename('Fraud - M9')

dfx = pd.merge(train_set_nof_m9, train_set_fraud_m9, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="M6 (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_nonfraud['M1'] = train_set_nonfraud['M1'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_nonfraud['M2'] = train_set_nonfraud['M2'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_nonfraud['M3'] = train_set_nonfraud['M3'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_nonfraud['M5'] = train_set_nonfraud['M5'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_nonfraud['M6'] = train_set_nonfraud['M6'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_nonfraud['M7'] = train_set_nonfraud['M7'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_nonfraud['M8'] = train_set_nonfraud['M8'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_nonfraud['M9'] = train_set_nonfraud['M9'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_nonfraud['Msum'] = train_set_nonfraud['M1'] + train_set_nonfraud['M2'] + train_set_nonfraud['M3'] + train_set_nonfraud['M5'] + train_set_nonfraud['M6'] + train_set_nonfraud['M7'] + train_set_nonfraud['M8'] + train_set_nonfraud['M9']

In [ ]:
train_set_fraud['M1'] = train_set_fraud['M1'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_fraud['M2'] = train_set_fraud['M2'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_fraud['M3'] = train_set_fraud['M3'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_fraud['M5'] = train_set_fraud['M5'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_fraud['M6'] = train_set_fraud['M6'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_fraud['M7'] = train_set_fraud['M7'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_fraud['M8'] = train_set_fraud['M8'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_fraud['M9'] = train_set_fraud['M9'].replace('T','1').replace('F','0').replace('None','0').fillna('0').astype(int)
train_set_fraud['Msum'] = train_set_fraud['M1'] + train_set_fraud['M2'] + train_set_fraud['M3'] + train_set_fraud['M5'] + train_set_fraud['M6'] + train_set_fraud['M7'] + train_set_fraud['M8'] + train_set_fraud['M9']

In [ ]:
train_set_nof_msum = train_set_nonfraud['Msum'].value_counts().rename('Non-Fraud - Msum')
train_set_fraud_msum = train_set_fraud['Msum'].value_counts().rename('Fraud - Msum')

dfx = pd.merge(train_set_nof_msum, train_set_fraud_msum, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="MSum (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})

In [ ]:
train_set_fraud['Dsum'].describe()

In [ ]:
train_set_nonfraud['Dsum'].describe()

In [ ]:
train_set.boxplot(by ='isFraud',column =['Dsum'], fontsize=25)
title_boxplot = 'Sum of Time Delta by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
train_set[['Dsum_minmax']] = minmax_scaler.fit_transform(train_set[['Dsum']])
test_set[['Dsum_minmax']] = minmax_scaler.fit_transform(test_set[['Dsum']])
#minmax_scaler.fit_transform(test_set['Dsum'])

In [ ]:
train_set.boxplot(by ='isFraud',column =['Dsum_minmax'], fontsize=25)
title_boxplot = 'Sum of Time Delta by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set.boxplot(by ='isFraud',column =['TransactionAmt'], fontsize=25)
title_boxplot = 'Transaction Amount by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set_nonfraud['TransactionAmt'].describe()

In [ ]:
train_set_fraud['TransactionAmt'].describe()

In [ ]:
minmax_scaler = MinMaxScaler()
train_set[['TransactionAmt_minmax']] = minmax_scaler.fit_transform(train_set[['TransactionAmt']])
test_set[['TransactionAmt_minmax']] = minmax_scaler.fit_transform(test_set[['TransactionAmt']])
#minmax_scaler.fit_transform(test_set['Dsum'])

In [ ]:
train_set.boxplot(by ='isFraud',column =['TransactionAmt_minmax'], fontsize=25)
title_boxplot = 'Transaction Amount by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set[(train_set['TransactionAmt']< 50)].boxplot(by ='isFraud',column =['TransactionAmt'], fontsize=25)
title_boxplot = 'Transaction Amount Less than $50 by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set[(train_set['TransactionAmt']>=50) & (train_set['TransactionAmt']<100)].boxplot(by ='isFraud',column =['TransactionAmt'], fontsize=25)
title_boxplot = 'Transaction Amount between $50 and $100 by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set[(train_set['TransactionAmt']>=100) & (train_set['TransactionAmt']<200)].boxplot(by ='isFraud',column =['TransactionAmt'], fontsize=25)
title_boxplot = 'Transaction Amount between $100 and $200 by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set_nof_amtcat = train_set_nonfraud['amt_cat'].value_counts().rename('Non-Fraud - Transaction Amount')
train_set_fraud_amtcat = train_set_fraud['amt_cat'].value_counts().rename('Fraud - Transaction Amount')

dfx = pd.merge(train_set_nof_amtcat, train_set_fraud_amtcat, right_index = True,left_index = True, how="outer")
dfx.plot.pie(subplots=True, figsize=(fig_width*5,fig_height*5), fontsize = font_size*2, autopct='%1.1f%%', title="Transaction Amount (Non-Fraud Vs Fraud)", legend=False)
plt.rcParams.update({'font.size': 20})


In [ ]:
train_set.boxplot(by ='isFraud',column =['Csum'], fontsize=25)
title_boxplot = 'Sum of Counting by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set_nonfraud['Csum'].describe()

In [ ]:
train_set_fraud['Csum'].describe()

In [ ]:
minmax_scaler = MinMaxScaler()
train_set[['Csum_minmax']] = minmax_scaler.fit_transform(train_set[['Csum']])
test_set[['Csum_minmax']] = minmax_scaler.fit_transform(test_set[['Csum']])


In [ ]:
train_set.boxplot(by ='isFraud',column =['Csum_minmax'], fontsize=25)
title_boxplot = 'Sum of Counting by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set['Csum_minmax'].describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
stcaler = StandardScaler()
train_set[['Csum_stcaler']] = stcaler.fit_transform(train_set[['Csum']])
test_set[['Csum_stcaler']] = stcaler.fit_transform(test_set[['Csum']])

In [ ]:
train_set.boxplot(by ='isFraud',column =['Csum_stcaler'], fontsize=25)
title_boxplot = 'Sum of Counting by Non-Fraud and Fraud'
plt.title(title_boxplot)
plt.suptitle('')
plt.show()

In [ ]:
train_set['Csum_stcaler'].describe()